## LLM Ops Stack  - LiteLLM Proxy + Langfuse 

## 1. Setup LiteLLM Proxy

## 2. Make LLM Requests to LiteLLM Proxy

### 2.1 Setup Client Side Variables 

### 2.2 Make your first LLM Request 

### 2.3 View Traces on Langfuse

### 2.4 Call Anthropic, Bedrock models 

## 3. Advanced - Set Langfuse Trace ID, Tags, Metadata 

## 